# RESEARCH NOTEBOOK --> MACDBB

In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
import pandas as pd
import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [3]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "ETH-USDT"
timeframe = "1h"
days = 30

In [4]:
# Get the candles
candles = await clob.get_candles_last_days(
    exchange, trading_pair, timeframe, days, from_trades=False)

2024-11-08 13:20:11,577 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x30f23b490>
2024-11-08 13:20:11,579 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x30f125180>, 1873188.102327708)]']
connector: <aiohttp.connector.TCPConnector object at 0x30f23b4c0>


In [6]:
candles.data.head(2)

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume
timestamp,,,,,,,,,,
2024-10-09 17:00:00,1728493200,2432.94,2439.64,2427.21,2433.1,84417.789,205465743.49735999,115657,41359.022,100661508.85636
2024-10-09 18:00:00,1728496800,2433.1,2436.46,2408,2414.24,196111.684,474714876.46208,212355,97864.118,236946057.58978999


In [8]:
candles.plot(type="candles")

In [14]:
# EMAs
ema_short = 8
ema_medium = 29
ema_long = 31

# MACD
macd_fast = 22
macd_slow = 36
macd_signal = 17

# ATR
atr_length = 11
atr_multiplier = 1.5

# Add indicators
candles_df = candles.data
candles_df.ta.macd(fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
candles_df.ta.atr(length=atr_length, append=True)
candles_df.ta.ema(length=ema_short, append=True)
candles_df.ta.ema(length=ema_medium, append=True)
candles_df.ta.ema(length=ema_long, append=True)
candles_df["long_atr_support"] = candles_df["close"].shift(1) - candles_df[f"ATRr_{atr_length}"] * atr_multiplier
candles_df["short_atr_resistance"] = candles_df["close"].shift(1) + candles_df[f"ATRr_{atr_length}"] * atr_multiplier

candles_df.tail(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume,MACD_22_36_17,MACDh_22_36_17,MACDs_22_36_17,ATRr_11,EMA_8,EMA_29,EMA_31,long_atr_support,short_atr_resistance
timestamp,,,,,,,,,,,,,,,,,,,
2024-11-08 12:00:00,1731067200,2915.27,2942,2910.01,2937.04,143442.992,419908855.69613999,233219,73584.306,215469624.04113001,39.64109007,-5.24036777,44.88145784,29.27087955,2917.40867313,2869.88656137,2864.1010523,2871.36368068,2959.17631932
2024-11-08 13:00:00,1731070800,2937.04,2955.3,2925.34,2947.51,170863.047,502250410.58117998,279841,87919.315,258540926.15821001,39.42218606,-4.85268603,44.27487209,29.33352686,2924.09785688,2875.06145728,2869.31411153,2893.03970971,2981.04029029
2024-11-08 14:00:00,1731074400,2947.51,2958,2908.26,2913.46,293993.484,862745933.99787998,404854,137787.614,404418665.45389003,37.92749371,-5.64211412,43.56960782,31.18866078,2921.73388868,2877.62136013,2872.07322956,2900.72700883,2994.29299117
2024-11-08 15:00:00,1731078000,2913.47,2946.98,2903.25,2937.59,324911.226,951085143.92508996,458760,158164.324,463106945.39265001,37.25220716,-5.61546726,42.86767442,32.32878253,2925.25746898,2881.61926945,2876.16802771,2864.96682621,2961.95317379
2024-11-08 16:00:00,1731081600,2937.6,2945.73,2919.77,2930.67,108538.775,318126201.17390001,131107,54584.417,160029821.75411001,36.26618549,-5.86799016,42.13417565,31.7498023,2926.46025365,2884.88931816,2879.57440098,2889.96529655,2985.21470345


In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, 
                    subplot_titles=(trading_pair, 'MACD'),
                    row_heights=[0.7, 0.3])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='OHLC'),
              row=1, col=1)

# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
              row=2, col=1)

# Add ATR support and resistance
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["long_atr_support"],
                         line=dict(color='#00FF00', width=2),
                         name='Long ATR Support'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["short_atr_resistance"],
                         line=dict(color='#FF0000', width=2),
                         name='Short ATR Resistance'), row=1, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200, height=800,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()

In [17]:
# Generate signal

candles_df["signal"] = 0
macdh = candles_df[f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"]
short_ema = candles_df[f"EMA_{ema_short}"]
medium_ema = candles_df[f"EMA_{ema_medium}"]
long_ema = candles_df[f"EMA_{ema_long}"]
close = candles_df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (close > short_ema) & (close > candles_df["long_atr_support"]) & (macdh > 0) 
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (close < short_ema) & (close < candles_df["short_atr_resistance"]) & (macdh < 0)
candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [20]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=('OHLC with BB', 'MACD', 'Signal'),
                    row_heights=[0.6, 0.2, 0.2])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='Candlesticks'),
              row=1, col=1)


# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
              row=2, col=1)

# Add ATR support and resistance
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["long_atr_support"],
                         line=dict(color='#00FF00', width=2),
                         name='Long ATR Support'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["short_atr_resistance"],
                         line=dict(color='#FF0000', width=2),
                         name='Short ATR Resistance'), row=1, col=1)

# Add the signal line
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white")),
              row=3, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1500, height=1000,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()


# CONCLUSION

In this notebook, we have implemented a strategy combining the MACD (Moving Average Convergence Divergence) indicator with Bollinger Bands. We've visualized these indicators along with the price data and generated signals based on their interactions. This approach provides a solid foundation for our trading strategy.
 
## Key components of our strategy include:
 1. MACD for trend identification
 2. Bollinger Bands for volatility measurement and potential reversal points
 3. A signal line derived from the combination of these indicators
 
 The next step is to backtest this strategy to evaluate its profitability and robustness. For this purpose, we have created a controller file named `macd_bb.py` in this folder. This file implements the logic we've developed here, allowing us to conduct comprehensive backtests in the subsequent notebook.